In [1]:
import os

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
assert os.environ["TAVILY_API_KEY"]

In [3]:
tools = [TavilySearchResults(max_results=1)]

In [4]:
prompt = hub.pull("hwchase17/react")

/home/mori/miniforge3/envs/openmm/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [5]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [6]:
# Choose the LLM to use
llm = ChatOpenAI(temperature=0.3, model="gpt-4o")

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

/tmp/ipykernel_19993/410987464.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.3, model="gpt-4o")


In [7]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2,
    return_intermediate_results=True,
)

In [8]:
result = agent_executor.invoke(
    {"input": "What is the diference between OpenMM and LAMMPS?"}
)



> Entering new AgentExecutor chain...
OpenMM and LAMMPS are both molecular dynamics simulation software packages, but they have different focuses and features. To provide a comprehensive comparison, I will outline the key differences between them:

1. **Purpose and Design**:
   - **OpenMM**: It is designed primarily for high-performance molecular dynamics simulations, with a focus on ease of use and flexibility. It is particularly well-suited for simulations of biomolecular systems and is often used in the context of drug discovery and protein dynamics.
   - **LAMMPS (Large-scale Atomic/Molecular Massively Parallel Simulator)**: It is a more general-purpose molecular dynamics code that can handle a wide variety of systems, including soft matter, solid-state materials, and biomolecules. It is highly extensible and can be used for simulations involving complex force fields and custom interactions.

2. **Programming and Extensibility**:
   - **OpenMM**: It is designed to be user-friendl

In [13]:
result

{'input': 'What is the diference between OpenMM and LAMMPS?',
 'output': 'OpenMM is a molecular dynamics toolkit optimized for biomolecular simulations and GPU acceleration, with a user-friendly Python API. LAMMPS is a highly flexible and scalable simulator suitable for a wide range of applications, particularly large-scale simulations on parallel computing architectures.'}

In [10]:
from datetime import datetime
from typing import Any

from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction, AgentFinish


class ThoughtVisualizer(BaseCallbackHandler):
    def __init__(self):
        self.thoughts = []
        self.current_thought = {}
        self.step_count = 0

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        self.step_count += 1

        thought_entry = {
            "step": self.step_count,
            "timestamp": datetime.now().strftime("%H:%M:%S"),
            "thought": action.log,  # 思考プロセス
            "action": action.tool,  # 使用したツール
            "action_input": action.tool_input,  # ツールへの入力
            "type": "action",
        }

        self.thoughts.append(thought_entry)

    def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> Any:
        self.step_count += 1

        thought_entry = {
            "step": self.step_count,
            "timestamp": datetime.now().strftime("%H:%M:%S"),
            "final_answer": finish.return_values["output"],
            "thought": finish.log,
            "type": "finish",
        }

        self.thoughts.append(thought_entry)

    def print_thoughts(self):
        for entry in self.thoughts:
            print(f"\n=== ステップ {entry['step']} ({entry['timestamp']}) ===")

            if "thought" in entry and entry["thought"]:
                print("\n" + entry["thought"])

            if entry["type"] == "action":
                print(f"\nツール: {entry['action']}")
                print(f"入力: {entry['action_input']}")

            if entry["type"] == "finish":
                print(f"\n最終回答: {entry['final_answer']}")

            print("\n" + "=" * 50)

In [11]:
visualizer = ThoughtVisualizer()
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2,
    return_intermediate_results=True,
    callbacks=[visualizer],
)
result = agent_executor.invoke(
    {"input": "What is the diference between OpenMM and LAMMPS?"},
)



> Entering new AgentExecutor chain...
OpenMM and LAMMPS are both software packages used for molecular dynamics simulations, but they have different features and use cases. I will provide a detailed comparison based on their characteristics.

1. **Purpose and Design**:
   - **OpenMM**: OpenMM is designed to be a high-performance toolkit for molecular simulations, particularly focused on biomolecular systems. It is user-friendly and provides a Python API, making it accessible for researchers who prefer scripting in Python. OpenMM is optimized for running on GPUs, which can significantly accelerate simulations.
   - **LAMMPS**: LAMMPS (Large-scale Atomic/Molecular Massively Parallel Simulator) is a highly flexible and scalable molecular dynamics simulator. It is designed to run efficiently on parallel computing architectures, making it suitable for large-scale simulations. LAMMPS supports a wide range of potential models and simulation types, from atomic to mesoscale systems.

2. **Prog

In [12]:
visualizer.thoughts

[{'step': 1,
  'timestamp': '13:55:03',
  'final_answer': 'OpenMM is a molecular dynamics toolkit optimized for biomolecular simulations and GPU acceleration, with a user-friendly Python API. LAMMPS is a highly flexible and scalable simulator suitable for a wide range of applications, particularly large-scale simulations on parallel computing architectures.',
  'thought': 'OpenMM and LAMMPS are both software packages used for molecular dynamics simulations, but they have different features and use cases. I will provide a detailed comparison based on their characteristics.\n\n1. **Purpose and Design**:\n   - **OpenMM**: OpenMM is designed to be a high-performance toolkit for molecular simulations, particularly focused on biomolecular systems. It is user-friendly and provides a Python API, making it accessible for researchers who prefer scripting in Python. OpenMM is optimized for running on GPUs, which can significantly accelerate simulations.\n   - **LAMMPS**: LAMMPS (Large-scale Atomi